In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
# 数据加载

df = pd.read_excel('./material_group2.xlsx')
df

,material_group_code,material_group_name,risk1,risk2,risk3,risk4,risk5,risk6,risk7,risk8,risk9,risk10,risk11,risk12,risk13,risk14,risk15,risk16,risk17,risk18,risk19,risk20,risk21,risk22,risk23,risk24,money1,money2,money3,money4,money5,money6,money7,money8,money9,risk_label,money_label,risk_score,money_score
0,1,材料组1,567,378,31,817,241,405,166,502,968,333,405,941,470,695,563,257,984,938,591,38,931,185,703,875,742,760,484,867,64,862,297,959,418,1,0,12984,5453
1,2,材料组2,782,354,858,191,729,597,742,963,301,555,992,695,672,13,556,102,421,122,211,60,965,436,369,144,804,445,97,79,347,509,304,196,276,0,0,11830,3057
2,3,材料组3,659,28,933,446,488,18,673,760,975,134,288,47,604,137,229,410,138,598,538,396,972,13,816,256,920,625,337,350,417,742,640,114,958,0,0,10556,5103
3,4,材料组4,517,248,18,787,383,432,484,789,603,332,242,976,677,471,966,941,806,162,104,403,957,488,901,69,62,105,166,561,176,318,235,157,440,1,0,12756,2220
4,5,材料组5,948,116,513,115,833,242,673,160,372,991,11,505,864,746,493,713,50,546,93,566,778,207,856,423,481,190,226,693,531,912,947,773,429,0,0,11814,5182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,材料组96,856,344,78,937,98,675,477,271,644,737,586,152,161,38,195,889,843,775,806,191,676,961,974,389,309,99,101,886,44,857,245,54,414,1,0,12753,3009
96,97,材料组97,767,389,398,20,552,366,494,675,536,788,577,569,81,853,849,944,147,774,756,780,882,15,699,32,249,406,535,666,576,800,809,538,126,1,0,12943,4705
97,98,材料组98,689,822,495,109,968,562,202,440,233,427,171,850,191,882,241,704,870,700,588,703,775,96,961,778,675,126,936,355,469,926,237,700,742,1,0,13457,5166
98,99,材料组99,724,343,750,766,377,378,463,373,568,299,346,87,643,23,322,400,104,690,394,652,293,995,860,403,764,762,552,287,803,647,407,622,427,0,0,11253,5271


In [2]:
risk_cols = ['risk' + str(i) for i in range(1,25)]
money_cols = ['money' + str(i) for i in range(1,10)]
x_risk = df[risk_cols]
y_risk = df['risk_label']
x_money = df[money_cols]
y_money = df['money_label']

In [3]:
# 数据集切分
from sklearn.model_selection import train_test_split
x_train_risk, x_test_risk, y_train_risk, y_test_risk = train_test_split(x_risk, y_risk, test_size=0.2, random_state=34)
x_train_money, x_test_money, y_train_money, y_test_money = train_test_split(x_money, y_money, test_size=0.2, random_state=34)

In [4]:
# 使用逻辑回归进行分类
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
model = LogisticRegression(C=1, penalty='l2', tol=0.0001, max_iter=100, class_weight='balance')
model.fit(x_train_risk,y_train_risk)
y_risk_pred = model.predict(x_test_risk)
print(accuracy_score(y_risk_pred,y_test_risk))
model.score(x_test_risk, y_test_risk)

0.75


0.75

In [5]:
# 对risk进行model评估
from sklearn.model_selection import cross_val_score
print(cross_val_score(model, x_risk, y_risk, cv = 5, scoring='accuracy' ).mean())
print(cross_val_score(model, x_money, y_money, cv = 5, scoring='accuracy' ).mean())

0.73
0.9299999999999999


In [6]:
def lr_risk_cv(max_iter, C):
    result = np.mean(cross_val_score(LogisticRegression(C=C, max_iter=max_iter), x_risk, y_risk, cv = 5, scoring='accuracy' ))
    return result

def lr_money_cv(max_iter, C):
    result = np.mean(cross_val_score(LogisticRegression(C=C, max_iter=max_iter), x_money, y_money, cv = 5, scoring='accuracy' ))
    return result

In [7]:
# 使用贝叶斯超参数优化
from bayes_opt import BayesianOptimization
lr_op = BayesianOptimization(
    f = lr_money_cv,
    pbounds = {'C':(0.01,10),
             'max_iter':(50,500)}
)
#lr_op.maximize(init_points=5, n_iter=25)
#print(lr_op.max)

In [8]:
# 使用WOE + LR 进行建模
def get_bin_features(df, q=5):
    columns = df.columns
    for column in columns:
        df['bin_'+column] = pd.qcut(df[column], q=q, duplicates='drop')
    return df

In [9]:
x_data = x_risk.copy()
y_data = y_risk.copy()
x_data = get_bin_features(x_data)

In [10]:
bin_cols = ['bin_risk' + str(i) for i in range(1,25)]
x_data['target'] = y_data
x_data

,risk1,risk2,risk3,risk4,risk5,risk6,risk7,risk8,risk9,risk10,risk11,risk12,risk13,risk14,risk15,risk16,risk17,risk18,risk19,risk20,risk21,risk22,risk23,risk24,bin_risk1,bin_risk2,bin_risk3,bin_risk4,bin_risk5,bin_risk6,bin_risk7,bin_risk8,bin_risk9,bin_risk10,bin_risk11,bin_risk12,bin_risk13,bin_risk14,bin_risk15,bin_risk16,bin_risk17,bin_risk18,bin_risk19,bin_risk20,bin_risk21,bin_risk22,bin_risk23,bin_risk24,target
0,567,378,31,817,241,405,166,502,968,333,405,941,470,695,563,257,984,938,591,38,931,185,703,875,"(414.0, 598.2]","(371.8, 530.8]","(10.999, 173.8]","(755.6, 993.0]","(3.999, 241.8]","(357.6, 516.4]","(1.999, 180.0]","(436.4, 680.6]","(770.6, 988.0]","(274.8, 445.6]","(366.8, 537.2]","(809.6, 994.0]","(341.6, 577.6]","(584.2, 815.0]","(377.2, 602.4]","(255.8, 440.4]","(828.2, 989.0]","(792.6, 991.0]","(584.4, 785.2]","(2.999, 206.0]","(864.8, 994.0]","(175.2, 444.8]","(700.6, 828.4]","(756.0, 996.0]",1
1,782,354,858,191,729,597,742,963,301,555,992,695,672,13,556,102,421,122,211,60,965,436,369,144,"(743.2, 972.0]","(178.2, 371.8]","(808.0, 995.0]","(12.999, 199.8]","(640.2, 813.2]","(516.4, 771.4]","(669.4, 807.6]","(802.4, 989.0]","(196.8, 365.6]","(445.6, 628.0]","(793.4, 992.0]","(594.4, 809.6]","(577.6, 774.0]","(9.999, 220.8]","(377.2, 602.4]","(12.999, 255.8]","(252.8, 461.2]","(20.999, 170.6]","(165.6, 383.0]","(2.999, 206.0]","(864.8, 994.0]","(175.2, 444.8]","(226.0, 532.2]","(0.999, 236.2]",0
2,659,28,933,446,488,18,673,760,975,134,288,47,604,137,229,410,138,598,538,396,972,13,816,256,"(598.2, 743.2]","(4.999, 178.2]","(808.0, 995.0]","(371.0, 525.0]","(438.0, 640.2]","(1.999, 216.6]","(669.4, 807.6]","(680.6, 802.4]","(770.6, 988.0]","(20.999, 274.8]","(218.8, 366.8]","(2.999, 232.0]","(577.6, 774.0]","(9.999, 220.8]","(187.6, 377.2]","(255.8, 440.4]","(5.999, 252.8]","(446.6, 629.8]","(383.0, 584.4]","(391.2, 577.2]","(864.8, 994.0]","(12.999, 175.2]","(700.6, 828.4]","(236.2, 387.0]",0
3,517,248,18,787,383,432,484,789,603,332,242,976,677,471,966,941,806,162,104,403,957,488,901,69,"(414.0, 598.2]","(178.2, 371.8]","(10.999, 173.8]","(755.6, 993.0]","(241.8, 438.0]","(357.6, 516.4]","(476.2, 669.4]","(680.6, 802.4]","(365.6, 609.0]","(274.8, 445.6]","(218.8, 366.8]","(809.6, 994.0]","(577.6, 774.0]","(405.0, 584.2]","(838.0, 991.0]","(843.4, 997.0]","(647.6, 828.2]","(20.999, 170.6]","(3.999, 165.6]","(391.2, 577.2]","(864.8, 994.0]","(444.8, 671.2]","(828.4, 997.0]","(0.999, 236.2]",1
4,948,116,513,115,833,242,673,160,372,991,11,505,864,746,493,713,50,546,93,566,778,207,856,423,"(743.2, 972.0]","(4.999, 178.2]","(415.0, 550.8]","(12.999, 199.8]","(813.2, 999.0]","(216.6, 357.6]","(669.4, 807.6]","(7.999, 230.6]","(365.6, 609.0]","(775.4, 991.0]","(4.999, 218.8]","(423.0, 594.4]","(774.0, 986.0]","(584.2, 815.0]","(377.2, 602.4]","(646.8, 843.4]","(5.999, 252.8]","(446.6, 629.8]","(3.999, 165.6]","(391.2, 577.2]","(672.6, 864.8]","(175.2, 444.8]","(828.4, 997.0]","(387.0, 554.0]",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,856,344,78,937,98,675,477,271,644,737,586,152,161,38,195,889,843,775,806,191,676,961,974,389,"(743.2, 972.0]","(178.2, 371.8]","(10.999, 173.8]","(755.6, 993.0]","(3.999, 241.8]","(516.4, 771.4]","(476.2, 669.4]","(230.6, 436.4]","(609.0, 770.6]","(628.0, 775.4]","(537.2, 793.4]","(2.999, 232.0]","(5.999, 197.4]","(9.999, 220.8]","(187.6, 377.2]","(843.4, 997.0]","(828.2, 989.0]","(629.8, 792.6]","(785.2, 972.0]","(2.999, 206.0]","(672.6, 864.8]","(815.0, 995.0]","(828.4, 997.0]","(387.0, 554.0]",1
96,767,389,398,20,552,366,494,675,536,788,577,569,81,853,849,944,147,774,756,780,882,15,699,32,"(743.2, 972.0]","(371.8, 530.8]","(173.8, 415.0]","(12.999, 199.8]","(438.0, 640.2]","(357.6, 516.4]","(476.2, 669.4]","(436.4, 680.6]","(365.6, 609.0]","(775.4, 991.0]","(537.2, 793.4]","(423.0, 594.4]","(5.999, 197.4]","(815.0, 989.0]","

In [11]:
# 计算IV，衡量自变量的预测能力
def cal_IV(df, feature, target):
    lst = []
    cols = ['feature', 'val', 'num', 'bad_num'] # 分别代表 字段名称、分箱数值段、在该分箱数值段的总个数、在该分箱数值中bad的个数
    for i in range(df[feature].nunique()): #nunique = unique的个数
        val = list(df[feature].unique())[i]
        # 统计feature，feature_value， 这个value的个数，这个value导致target为1的个数
        lst.append([feature, val])
        temp1 = df[df[feature]==val].count()[feature] # 这个value的总个数
        temp2 = df[(df[feature]==val) & (df[target]==1)].count()[feature] # 这个value导致target为1的个数
        #print(feature, val, temp1, temp2)
        lst.append([feature, val, temp1, temp2])
    data = pd.DataFrame(lst, columns=cols)
    data = data[data['bad_num']>0]
    data['num_ratio'] = data['num'] / data['num'].sum()
    data['bad_ratio'] = data['bad_num'] / data['num']
    data['bad_margin_rate'] = data['bad_num'] / data['bad_num'].sum()
    data['good_margin_rate'] = (data['num'] - data['bad_num'])/(data['num'].sum() - data['bad_num'].sum())
    data['WOE'] = np.log(data['bad_margin_rate']/data['good_margin_rate'])
    data['IV'] = data['WOE'] * (data['bad_margin_rate'] - data['good_margin_rate'])
    
    data = data.sort_values(by=['feature', 'val'])
    return data['IV'].sum()

In [12]:
# 计算df中bin字段的IV值， target为目标列
def cal_all_IV(df, target):
    # 统计所有字段的IV值
    result = {}
    for col in bin_cols:
        temp_IV = cal_IV(df, col, target)
        result[col] = temp_IV
    return result

In [13]:
# 计算各个bin字段的IV值
col_IVs = cal_all_IV(x_data, 'target')
col_IVs

{'bin_risk1': 0.17727283444622716,
 'bin_risk2': 0.29756118013103733,
 'bin_risk3': 0.23268753967718636,
 'bin_risk4': 0.2002258090738322,
 'bin_risk5': 0.17727283444622716,
 'bin_risk6': 0.4231252394148584,
 'bin_risk7': 0.2245463207508226,
 'bin_risk8': 0.12437347794540737,
 'bin_risk9': 0.1908782377537943,
 'bin_risk10': 0.2770849297737586,
 'bin_risk11': 0.1070491184427093,
 'bin_risk12': 0.41962027297186977,
 'bin_risk13': 0.04207073573306754,
 'bin_risk14': 0.04609085064345349,
 'bin_risk15': 0.7441720867209729,
 'bin_risk16': 0.1908782377537943,
 'bin_risk17': 0.3352953508301903,
 'bin_risk18': 0.36016199013179984,
 'bin_risk19': 0.2437775942546141,
 'bin_risk20': 0.09680619948142206,
 'bin_risk21': 0.4038939659110669,
 'bin_risk22': 0.12437347794540737,
 'bin_risk23': 0.009647629589526495,
 'bin_risk24': 0.3727352132122833}

In [14]:
# 筛选IV值大于0.1的bin字段名称
def col_filtered(col_IVs, threshold):
    result = {k:v for k, v in col_IVs.items() if v >= threshold}
    return result.keys()

In [15]:
col_filter = col_filtered(col_IVs, 0.1)
col_filter

dict_keys(['bin_risk1', 'bin_risk2', 'bin_risk3', 'bin_risk4', 'bin_risk5', 'bin_risk6', 'bin_risk7', 'bin_risk8', 'bin_risk9', 'bin_risk10', 'bin_risk11', 'bin_risk12', 'bin_risk15', 'bin_risk16', 'bin_risk17', 'bin_risk18', 'bin_risk19', 'bin_risk21', 'bin_risk22', 'bin_risk24'])

In [16]:
# 计算字段的WOE特征值
def cal_WOE(df, features, target):
    for feature in features:
        df_woe = df.groupby(feature).agg({target:['sum', 'count']})
        df_woe.columns = list(map(''.join, df_woe.columns.values))
        #print(df_woe.columns)
        df_woe = df_woe.reset_index()
        df_woe = df_woe.rename(columns={target+'sum': 'bad_num', target+'count': 'num'})
        #print(df_woe)
       
        df_woe['good_num'] = df_woe['num'] - df_woe['bad_num']
        df_woe = df_woe[[feature, 'good_num', 'bad_num']]
        
        df_woe['bad_margin_rate'] = df_woe['bad_num'] / df_woe['bad_num'].sum()
        df_woe['good_margin_rate'] = df_woe['good_num'] / df_woe['good_num'].sum()
        
        #计算woe
        df_woe['woe'] = np.log1p(df_woe['bad_margin_rate']/df_woe['good_margin_rate'])
        # 在后面拼接上 _feature, 比如_age
        df_woe.columns = [c if c == feature else c + '_' + feature for c in list(df_woe.columns.values)]
        # 拼接
        df = df.merge(df_woe, on=feature, how='left')
    return df

In [17]:
# 计算过滤后特征的WOE
df_woe = cal_WOE(x_data, col_filter, 'target')
df_woe

,risk1,risk2,risk3,risk4,risk5,risk6,risk7,risk8,risk9,risk10,risk11,risk12,risk13,risk14,risk15,risk16,risk17,risk18,risk19,risk20,risk21,risk22,risk23,risk24,bin_risk1,bin_risk2,bin_risk3,bin_risk4,bin_risk5,bin_risk6,bin_risk7,bin_risk8,bin_risk9,bin_risk10,bin_risk11,bin_risk12,bin_risk13,bin_risk14,bin_risk15,bin_risk16,bin_risk17,bin_risk18,bin_risk19,bin_risk20,bin_risk21,bin_risk22,bin_risk23,bin_risk24,target,good_num_bin_risk1,bad_num_bin_risk1,bad_margin_rate_bin_risk1,good_margin_rate_bin_risk1,woe_bin_risk1,good_num_bin_risk2,bad_num_bin_risk2,bad_margin_rate_bin_risk2,good_margin_rate_bin_risk2,woe_bin_risk2,good_num_bin_risk3,bad_num_bin_risk3,bad_margin_rate_bin_risk3,good_margin_rate_bin_risk3,woe_bin_risk3,good_num_bin_risk4,bad_num_bin_risk4,bad_margin_rate_bin_risk4,good_margin_rate_bin_risk4,woe_bin_risk4,good_num_bin_risk5,bad_num_bin_risk5,bad_margin_rate_bin_risk5,good_margin_rate_bin_risk5,woe_bin_risk5,good_num_bin_risk6,bad_num_bin_risk6,bad_margin_rate_bin_risk6,good_margin_rate_bin_risk6,woe_bin_risk6,good_num_bin_risk7,bad_num_bin_risk7,bad_margin_rate_bin_risk7,good_margin_rate_bin_risk7,woe_bin_risk7,good_num_bin_risk8,bad_num_bin_risk8,bad_margin_rate_bin_risk8,good_margin_rate_bin_risk8,woe_bin_risk8,good_num_bin_risk9,bad_num_bin_risk9,bad_margin_rate_bin_risk9,good_margin_rate_bin_risk9,woe_bin_risk9,good_num_bin_risk10,bad_num_bin_risk10,bad_margin_rate_bin_risk10,good_margin_rate_bin_risk10,woe_bin_risk10,good_num_bin_risk11,bad_num_bin_risk11,bad_margin_rate_bin_risk11,good_margin_rate_bin_risk11,woe_bin_risk11,good_num_bin_risk12,bad_num_bin_risk12,bad_margin_rate_bin_risk12,good_margin_rate_bin_risk12,woe_bin_risk12,good_num_bin_risk15,bad_num_bin_risk15,bad_margin_rate_bin_risk15,good_margin_rate_bin_risk15,woe_bin_risk15,good_num_bin_risk16,bad_num_bin_risk16,bad_margin_rate_bin_risk16,good_margin_rate_bin_risk16,woe_bin_risk16,good_num_bin_risk17,bad_num_bin_risk17,bad_margin_rate_bin_risk17,good_margin_rate_bin_risk17,woe_bin_risk17,good_num_bin_risk18,bad_num_bin_risk18,bad_margin_rate_bin_risk18,good_margin_rate_bin_risk18,woe_bin_risk18,good_num_bin_risk19,bad_num_bin_risk19,bad_margin_rate_bin_risk19,good_margin_rate_bin_risk19,woe_bin_risk19,good_num_bin_risk21,bad_num_bin_risk21,bad_margin_rate_bin_risk21,good_margin_rate_bin_risk21,woe_bin_risk21,good_num_bin_risk22,bad_num_bin_risk22,bad_margin_rate_bin_risk22,good_margin_rate_bin_risk22,woe_bin_risk22,good_num_bin_risk24,bad_num_bin_risk24,bad_margin_rate_bin_risk24,good_margin_rate_bin_risk24,woe_bin_risk24
0,567,378,31,817,241,405,166,502,968,333,405,941,470,695,563,257,984,938,591,38,931,185,703,875,"(414.0, 598.2]","(371.8, 530.8]","(10.999, 173.8]","(755.6, 993.0]","(3.999, 241.8]","(357.6, 516.4]","(1.999, 180.0]","(436.4, 680.6]","(770.6, 988.0]","(274.8, 445.6]","(366.8, 537.2]","(809.6, 994.0]","(341.6, 577.6]","(584.2, 815.0]","(377.2, 602.4]","(255.8, 440.4]","(828.2, 989.0]","(792.6, 991.0]","(584.4, 785.2]","(2.999, 206.0]","(864.8, 994.0]","(175.2, 444.8]","(700.6, 828.4]","(756.0, 996.0]",1,9,11,0.211538,0.1875,0.755279,14,6,0.115385,0.291667,0.333328,11,9,0.173077,0.229167,0.562608,9,11,0.211538,0.187500,0.755279,12,8,0.153846,0.250000,0.479573,6,14,0.269231,0.125000,1.148623,13,7,0.134615,0.270833,0.403491,7,13,0.250000,0.145833,0.998529,7,13,0.250000,0.145833,0.998529,11,9,0.173077,0.229167,0.562608,8,12,0.230769,0.166667,0.869038,6,14,0.269231,0.125000,1.148623,6,14,0.269231,0.125000,1.148623,13,7,0.134615,0.270833,0.403491,8,12,0.230769,0.166667,0.869038,7,13,0.250000,0.145833,0.998529,6,14,0.269231,0.125000,1.148623,8,12,0.230769,0.166667,0.869038,12,8,0.153846,0.250000,0.479573,5,15,0.288462,0.104167,1.326871
1,782,354,858,191,729,597,742,963,301,555,992,695,672,13,556,102,421,122,211,60,965,436,369,144,"(743.2, 972.0]","(178.2, 371.8]","(808.0, 995.0]","(12.999, 199.8]","(640.2, 813.2]","(516.4, 771.4]","(669.4, 807.6]","(802.4, 989.0]","(196.8, 365.6]","(445.6, 628.0]","(793.4, 992.0]","(594.4, 809.6

In [18]:
# 得到woe字段
woe_cols = [i for i in list(df_woe.columns.values) if 'woe' in i]
df_woe[woe_cols]

,woe_bin_risk1,woe_bin_risk2,woe_bin_risk3,woe_bin_risk4,woe_bin_risk5,woe_bin_risk6,woe_bin_risk7,woe_bin_risk8,woe_bin_risk9,woe_bin_risk10,woe_bin_risk11,woe_bin_risk12,woe_bin_risk15,woe_bin_risk16,woe_bin_risk17,woe_bin_risk18,woe_bin_risk19,woe_bin_risk21,woe_bin_risk22,woe_bin_risk24
0,0.755279,0.333328,0.562608,0.755279,0.479573,1.148623,0.403491,0.998529,0.998529,0.562608,0.869038,1.148623,1.148623,0.403491,0.869038,0.998529,1.148623,0.869038,0.479573,1.326871
1,1.148623,0.653926,1.098612,0.403491,0.562608,0.869038,0.562608,0.755279,0.755279,0.998529,0.755279,1.148623,1.148623,0.755279,0.562608,0.755279,0.653926,0.869038,0.479573,0.755279
2,0.479573,0.653926,1.098612,1.045969,0.755279,0.562608,0.562608,0.755279,0.998529,0.333328,0.479573,0.333328,0.403491,0.403491,0.403491,0.403491,0.403491,0.869038,0.755279,0.562608
3,0.755279,0.653926,0.562608,0.755279,0.653926,1.148623,0.998529,0.755279,0.869038,0.562608,0.479573,1.148623,1.098612,0.998529,1.326871,0.755279,0.562608,0.869038,0.755279,0.755279
4,1.148623,0.653926,0.653926,0.403491,1.148623,0.268264,0.562608,0.479573,0.869038,0.869038,0.562608,0.604736,1.148623,0.869038,0.403491,0.403491,0.562608,0.998529,0.479573,0.333328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.148623,0.653926,0.562608,0.755279,0.479573,0.869038,0.998529,0.562608,0.403491,0.869038,0.869038,0.333328,0.403491,0.998529,0.869038,1.148623,0.869038,0.998529,0.998529,0.333328
96,1.148623,0.333328,0.403491,0.403491,0.755279,1.148623,0.998529,0.998529,0.869038,0.869038,0.869038,0.604736,1.098612,0.998529,0.403491,1.148623,1.148623,0.869038,0.755279,0.755279
97,0.479573,0.869038,0.653926,0.403491,1.148623,0.869038,0.653926,0.998529,0.755279,0.562608,0.562608,1.148623,0.403491,0.869038,0.869038,1.148623,1.148623,0.998529,0.755279,1.326871
98,0.479573,0.653926,0.916291,0.755279,0.653926,1.148623,0.653926,0.562608,0.869038,0.562608,0.479573,0.333328,0.403491,0.403491,0.403491,1.148623,0.403491,0.562608,0.998529,0.333328


In [19]:
feature_cols = [x[4:] for x in col_filter]
feature_cols

['risk1',
 'risk2',
 'risk3',
 'risk4',
 'risk5',
 'risk6',
 'risk7',
 'risk8',
 'risk9',
 'risk10',
 'risk11',
 'risk12',
 'risk15',
 'risk16',
 'risk17',
 'risk18',
 'risk19',
 'risk21',
 'risk22',
 'risk24']

In [20]:
# 得到WOE规则
def get_woe_rules(df_woe):
    
    df_bin_to_woe = pd.DataFrame(columns=['features', 'bin', 'woe'])
    for f in feature_cols:
        b =  'bin_' + f 
        w = 'woe_bin_' + f
        df = df_woe[[w, b]].drop_duplicates()
        df.columns = ['woe', 'bin']
        df['features'] = f
        df = df[['features', 'bin', 'woe']]
        df_bin_to_woe = pd.concat([df_bin_to_woe, df])
    return df_bin_to_woe
df_bin_to_woe = get_woe_rules(df_woe)
df_bin_to_woe

,features,bin,woe
0,risk1,"(414.0, 598.2]",0.755279
1,risk1,"(743.2, 972.0]",1.148623
2,risk1,"(598.2, 743.2]",0.479573
7,risk1,"(0.999, 249.4]",0.562608
9,risk1,"(249.4, 414.0]",0.653926
...,...,...,...
0,risk24,"(756.0, 996.0]",1.326871
1,risk24,"(0.999, 236.2]",0.755279
2,risk24,"(236.2, 387.0]",0.562608
4,risk24,"(387.0, 554.0]",0.333328


# 逻辑回归建模

In [21]:
x_train, x_test, y_train, y_test = train_test_split(df_woe[woe_cols], df_woe['target'], test_size=0.2, random_state=33)
print('bad rate is ',y_train.mean())

bad rate is  0.5


In [22]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=33,class_weight='balanced', max_iter=500).fit(x_train, y_train)
print(model.score(x_test,y_test))

from sklearn.metrics import f1_score, roc_auc_score
y_pred = model.predict(x_test)
print(f1_score(y_pred, y_test))
print(roc_auc_score(y_pred, y_test))

0.75
0.7368421052631579
0.8076923076923077


In [23]:
# 求评分卡模型的A,B
# A = P0 + B*ln(θ0)
# B = PDO/ln2
# 假设odds=1的时候，特定分数为650

A = 650
B = 20

def generate_scorecard(model_coef, binning_df, features, B):
    lst=[]
    cols = ['Variable', 'Binning', 'Score']
    # 模型系数
    coef = model_coef[0]
    for i in range(len(features)):
        f = features[i]
        # 得到这个feature的WOE规则
        df = binning_df[binning_df['features'] == f]
        for index, row in df.iterrows():
            lst.append([f, row['bin'], int(round(-coef[i] * row['woe'] * B))])
    data = pd.DataFrame(lst, columns=cols)
    return data

#
score_card = generate_scorecard(model.coef_, df_bin_to_woe, feature_cols, B)
score_card

,Variable,Binning,Score
0,risk1,"(414.0, 598.2]",-8
1,risk1,"(743.2, 972.0]",-13
2,risk1,"(598.2, 743.2]",-5
3,risk1,"(0.999, 249.4]",-6
4,risk1,"(249.4, 414.0]",-7
...,...,...,...
95,risk24,"(756.0, 996.0]",-36
96,risk24,"(0.999, 236.2]",-20
97,risk24,"(236.2, 387.0]",-15
98,risk24,"(387.0, 554.0]",-9


In [24]:
# 对于每个字段按照降序排序，查看规则
sorted_scorecard = score_card.groupby('Variable').apply(lambda x:x.sort_values('Score', ascending=False))
sorted_scorecard

Variable         Binning  Score
Variable                                   
risk1    2     risk1  (598.2, 743.2]     -5
         3     risk1  (0.999, 249.4]     -6
         4     risk1  (249.4, 414.0]     -7
         0     risk1  (414.0, 598.2]     -8
         1     risk1  (743.2, 972.0]    -13
...              ...             ...    ...
risk9    43    risk9  (609.0, 770.6]     -6
         44    risk9  (7.999, 196.8]     -8
         41    risk9  (196.8, 365.6]    -10
         42    risk9  (365.6, 609.0]    -12
         40    risk9  (770.6, 988.0]    -14

[100 rows x 3 columns]

In [25]:
# 把数据映射到分箱中
def str2int(s):
    if s == '-inf':
        return -999999
    if s == 'inf':
        return 999999
    return float(s)

# 将value映射到bin
def map_value2bin(feature_value, feature2bin):
    for index, row in feature2bin.iterrows():
        bins = str(row['Binning'])
        left_open = bins[0]=='('
        right_open = bins[-1] == ')'
        binnings = bins[1:-1].split(',')
        in_range = True
        temp = str2int(binnings[0])
        temp2 = str2int(binnings[1])
        # 检查左括号
        if left_open:
            if feature_value <= temp:
                in_range = False
        else:
            if feature_value < temp:
                in_range = False
        # 检查右括号
        if right_open:
            if feature_value >= temp2:
                in_range = False
        else:
            if feature_value > temp2:
                in_range = False
        if in_range:
            return row['Binning']
        

In [26]:
# df为待转换样本， scorecard为评分卡规则
def map2score(df, score_card):
    scored_columns = list(score_card['Variable'].unique())
    score = 0
    for col in scored_columns:
        # 取出评分规则
        feature2bin = score_card[score_card['Variable'] == col]
        # 取出具体的feature_value
        feature_value = df[col]
        selected_bin = map_value2bin(feature_value, feature2bin)
        temp_score = feature2bin[feature2bin['Binning'] == selected_bin]
        score += temp_score['Score'].values[0]
    return score

In [27]:
def cal_score(df, score_card, A):
    # map2score 按评分卡规则进行计算
    df['Score'] = df.apply(map2score, args=(score_card,), axis = 1)
    df['Score'] = df['Score'].astype(int)
    df['Score'] = df['Score'] + A
    return df

In [28]:
# 随机选择Good的5个人
good_sample = x_data[x_data['target'] == 0].sample(5)
good_sample = good_sample[feature_cols]
# 对5个好人进行评分
cal_score(good_sample, score_card, A)

,risk1,risk2,risk3,risk4,risk5,risk6,risk7,risk8,risk9,risk10,risk11,risk12,risk15,risk16,risk17,risk18,risk19,risk21,risk22,risk24,Score
2,659,28,933,446,488,18,673,760,975,134,288,47,229,410,138,598,538,972,13,256,444
5,759,166,140,36,331,976,673,434,701,348,110,594,367,842,921,24,706,58,670,452,444
29,725,729,470,30,40,319,896,438,721,777,837,19,49,146,138,113,825,151,737,498,469
12,695,67,321,284,382,244,816,494,180,275,826,723,890,514,369,926,725,253,110,23,418
38,55,695,800,967,498,692,59,690,104,728,778,105,562,646,96,161,238,596,394,284,413


In [30]:
# 随机选择Bad的5个人
bad_sample = x_data[x_data['target'] == 1].sample(5)
bad_sample = bad_sample[feature_cols]
# 对5个坏人进行评分
cal_score(bad_sample, score_card, A)

,risk1,risk2,risk3,risk4,risk5,risk6,risk7,risk8,risk9,risk10,risk11,risk12,risk15,risk16,risk17,risk18,risk19,risk21,risk22,risk24,Score
41,292,994,49,154,637,838,828,185,660,385,799,606,345,904,318,25,963,775,332,587,410
93,394,289,49,499,897,437,711,989,295,638,71,299,170,646,321,152,931,876,689,858,395
19,522,520,18,828,202,777,613,768,652,450,5,994,256,46,316,93,852,348,836,750,414
86,495,31,808,307,569,327,667,989,490,643,392,97,556,886,846,900,15,883,928,793,379
23,857,100,324,832,584,458,453,867,777,931,897,315,535,163,848,710,833,974,144,244,382


In [31]:
pd.set_option('display.max_rows', None)
score_card

,Variable,Binning,Score
0,risk1,"(414.0, 598.2]",-8
1,risk1,"(743.2, 972.0]",-13
2,risk1,"(598.2, 743.2]",-5
3,risk1,"(0.999, 249.4]",-6
4,risk1,"(249.4, 414.0]",-7
5,risk2,"(371.8, 530.8]",-5
6,risk2,"(178.2, 371.8]",-11
7,risk2,"(4.999, 178.2]",-11
8,risk2,"(757.4, 994.0]",-14
9,risk2,"(530.8, 757.4]",-19
